# Calculating integrals over GTOs

In [12]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=8, linewidth=120)

In this example, we'll go over the high- and low-level machinery that GQCP offers in order to calculate integrals over Cartesian GTOs. We assume that you're familiar with the mathematical concepts of a _scalar basis_, a _shell_, a _basis function_, and a _primitive_. If you're not, here's a very succinct overview:
- Spin-orbitals are expanded in an underlying scalar basis.
- In order to compactify scalar bases, we use shells that group primitives according to their angular momentum.
- Therefore, in every shell, a _set_ of basis functions (of the same angular momentum) is implicitly defined.
- Every basis function is defined as a contraction (i.e. a linear combination, where the coefficients are called _contraction coefficients_) of primitives.
- Here, the primitives are Cartesian GTOs.

We'll start off by setting up a small molecular system and a scalar basis. Since we don't want to manually read in a basis set, we'll use `RSpinOrbitalBasis`'s functionality to provide us with a scalar basis.

In [13]:
molecule = gqcpy.Molecule([
    gqcpy.Nucleus(1, 0.0, 0.0, 0.0),
    gqcpy.Nucleus(1, 0.0, 0.0, 1.0)
])

In [14]:
spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
scalar_basis = spin_orbital_basis.scalarBasis()
shell_set = scalar_basis.shellSet()  # A shell set is just a collection of shells.

## Primitive engines

In GQCP, we define an _engine_ to be a computational object that is able to calculate integrals over _shells_, while a _primitive engine_ is defined to be a computational entity that can calculate integrals over _primitives_. 

In this example, we're taking the expansion of contracted GTOs in terms of their primitives for granted (using the implementations provided by the combination of `FunctionalOneElectronIntegralEngine` and `IntegralCalculator`), and we'll be focusing on calculating integrals over _primitives_.

## Overlap integrals

Let's get straight into it. Using the McMurchie-Davidson integral scheme, we can calculate the overlap integral over two primitives as follows.

In [32]:
def overlap_function_1D(a, K, i, b, L, j):
    
    # Negative Cartesian exponents should be ignored: the correct value for the corresponding integral is 0.
    if (i < 0) or (j < 0):
        return 0.0
    
    
    # Use the McMurchie-Davidson recursion to calculate the overlap integral.
    p = a + b
    E = gqcpy.McMurchieDavidsonCoefficient(K, a, L, b)
    
    return np.power(np.pi / p, 0.5) * E(i, j, 0)

In [33]:
def overlap_function(left, right):
    
    primitive_integral = 1.0
    
    # The overlap integral is separable in its three Cartesian components.
    for direction in [gqcpy.CartesianDirection.x, gqcpy.CartesianDirection.y, gqcpy.CartesianDirection.z]:
        i = left.cartesianExponents().value(direction)
        j = right.cartesianExponents().value(direction)
        
        a = left.gaussianExponent()
        b = right.gaussianExponent()

        K = left.center()[direction]
        L = right.center()[direction]

        primitive_integral *= overlap_function_1D(a, K, i, b, L, j)

    return primitive_integral

We'll wrap this function in to a `FunctionalPrimitiveEngine`, and then supply it as the primitive engine that should be used in a `FunctionalOneElectronIntegralEngine`. We're doing this in order to use GQCP's internal handling of the shells and contractions through the `IntegralCalculator.calculate` call.

In [34]:
primitive_overlap_engine = gqcpy.FunctionalPrimitiveIntegralEngine_d(overlap_function)
overlap_engine = gqcpy.FunctionalOneElectronIntegralEngine_d(primitive_overlap_engine)

In [35]:
S = gqcpy.IntegralCalculator.calculate(overlap_engine, shell_set, shell_set)
print(S)

[[0.99999999 0.79658829]
 [0.79658829 0.99999999]]


We can verify our results by letting the spin-orbital basis quantize the overlap operator.

In [36]:
S_ref = spin_orbital_basis.quantizeOverlapOperator().parameters()
print(S_ref)

[[0.99999999 0.79658829]
 [0.79658829 0.99999999]]


## Kinetic integrals

In [42]:
def kinetic_function_1D(a, K, i, b, L, j):
    
    # The kinetic 1D integral is a sum of three 1D overlap integrals.
    return -2.0 * np.power(b, 2) * overlap_function_1D(a, K, i, b, L, j + 2) + \
           b * (2 * j + 1) * overlap_function_1D(a, K, i, b, L, j) - \
           0.5 * j * (j - 1) * overlap_function_1D(a, K, i, b, L, j - 2)

In [43]:
def kinetic_function(left, right):
    
    # Prepare some variables
    i = left.cartesianExponents().value(gqcpy.CartesianDirection.x)
    k = left.cartesianExponents().value(gqcpy.CartesianDirection.y)
    m = left.cartesianExponents().value(gqcpy.CartesianDirection.z)
    
    j = right.cartesianExponents().value(gqcpy.CartesianDirection.x)
    l = right.cartesianExponents().value(gqcpy.CartesianDirection.y)
    n = right.cartesianExponents().value(gqcpy.CartesianDirection.z)
    
    a = left.gaussianExponent()
    b = right.gaussianExponent()

    K_x = left.center()[gqcpy.CartesianDirection.x]
    K_y = left.center()[gqcpy.CartesianDirection.y]
    K_z = left.center()[gqcpy.CartesianDirection.z]
    
    L_x = right.center()[gqcpy.CartesianDirection.x]
    L_y = right.center()[gqcpy.CartesianDirection.y]
    L_z = right.center()[gqcpy.CartesianDirection.z]
    
    
    # The 3D kinetic energy integral is a sum of three contributions (dx^2, dy^2, dz^2).
    return kinetic_function_1D(a, K_x, i, b, L_x, j) * \
               overlap_function_1D(a, K_y, k, b, L_y, l) * \
               overlap_function_1D(a, K_z, m, b, L_z, n) + \
           overlap_function_1D(a, K_x, i, b, L_x, j) * \
               kinetic_function_1D(a, K_y, k, b, L_y, l) * \
               overlap_function_1D(a, K_z, m, b, L_z, n) + \
           overlap_function_1D(a, K_x, i, b, L_x, j) * \
               overlap_function_1D(a, K_y, k, b, L_y, l) * \
               kinetic_function_1D(a, K_z, m, b, L_z, n);

Same thing for the kinetic integrals. We wrap the `kinetic_function` in a functional primitive engine and proceed analoguously.

In [44]:
primitive_kinetic_engine = gqcpy.FunctionalPrimitiveIntegralEngine_d(kinetic_function)
kinetic_engine = gqcpy.FunctionalOneElectronIntegralEngine_d(primitive_kinetic_engine)

In [45]:
T = gqcpy.IntegralCalculator.calculate(kinetic_engine, shell_set, shell_set)
print(T)

[[0.76003188 0.38325367]
 [0.38325367 0.76003188]]


As a comparison, here are the GQCP integrals.

In [46]:
T_ref = spin_orbital_basis.quantizeKineticOperator().parameters()
print(T_ref)

[[0.76003188 0.38325367]
 [0.38325367 0.76003188]]


## Nuclear attraction integrals

In [ ]:
def nuclear_attraction_function(left, right):

## Coulomb repulsion integrals

In [ ]:
def coulomb_repulsion_function(left, right):
    